In [8]:
from common import *

cursor = connect()

# Event trigger

每当关联表上发生 `INSERT`、`UPDATE`、`DELETE` 或 `TRUNCATE` 事件时，就会触发常规触发器。要自动响应与数据定义语言 (DDL) 语句相关的事件，可以使用事件触发器。PostgreSQL 支持以下事件：
- ddl_command_start
- ddl_command_end
- table_rewrite
- sql_drop

语法
```
CREATE OR REPLACE FUNCTION event_trigger_function_name()
RETURNS EVENT_TRIGGER
AS
$$
BEGIN
   -- trigger logic
   -- ...
   -- no RETURN statement
END;
$$;
```
```
CREATE EVENT TRIGGER trigger_name
ON event
EXECUTE FUNCTION event_trigger_function_name()
```


## ddl_command_start

 `ddl_command_start` 事件发生在 PostgreSQL 执行 `CREATE`, `ALTER`, `DROP`, `GRANT`, `REVOKE`, `SECURITY` `LABEL`, 和 `COMMENT` 语句之前。完整的支持请查看[此链接](https://www.postgresql.org/docs/current/event-trigger-matrix.html)。
 对于 `databases`、`tablespaces` 和 `roles` 色等共享对象，不会发生 `ddl_command_start` 事件。
 
## ddl_command_end

`ddl_command_end` 事件发生在上述 DDL 语句之后。

## table_rewrite

`table_rewrite` 事件发生在 `ALTER TABLE` 或 `ALTER TYPE` 语句之前。

## sql_drop

`sql_drop` 事件发生在删除 databse 对象之后，在 `ddl_command_end` 事件之前。

In [2]:
sql = """
CREATE TABLE audits (
    id SERIAL PRIMARY KEY,
    username VARCHAR(100) NOT NULL,
    event VARCHAR(50) NOT NULL,
    command TEXT NOT NULL,
    executed_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);


CREATE OR REPLACE FUNCTION audit_command()
RETURNS EVENT_TRIGGER 
AS $$
BEGIN
    INSERT INTO audits (username, event , command)
    VALUES (session_user, TG_EVENT, TG_TAG );
END;
$$ LANGUAGE plpgsql;


CREATE EVENT TRIGGER audit_ddl_commands
ON ddl_command_end
EXECUTE FUNCTION audit_command();
"""
cursor.execute(sql)

<psycopg.Cursor [COMMAND_OK] [INTRANS] (host=localhost user=postgres database=dvdrental) at 0x1b4ffb193d0>

In [3]:
sql = """
CREATE TABLE regions(
    id SERIAL PRIMARY KEY,
    name VARCHAR(255) NOT NULL   
);
"""
cursor.execute(sql)

<psycopg.Cursor [COMMAND_OK] [INTRANS] (host=localhost user=postgres database=dvdrental) at 0x1b4ffb193d0>

In [4]:
sql = """
SELECT * FROM audits;
"""
run_sql(cursor, sql)

   id  username            event       command                executed_at
0   1  postgres  ddl_command_end  CREATE TABLE 2024-04-24 07:51:45.707484


# 普通触发器与事件触发器

| 特性 | 普通触发器 | 事件触发器 |
| --- | --- | --- |
|触发器等级| Table-level trigger, associated with a specific table and fired on INSERT, UPDATE, DELETE, or TRUNCATE statement. | Database-level triggers fired in response to DDL statements such as CREATE, ALTER, DROP, etc. |
|执行时机|Can be fired BEFORE, AFTER, or INSTEAD OF DML operations|Fired at some events including ddl_command_start, ddl_command_end, table_rewrite, sql_drop
|访问的数据|Has access to the data being modified|Has access to metadata
|使用场景|Logging changes to a specific table, updating related tables, and enforcing business rules.|Auditing DDL commands, and monitoring user activities.


# 条件触发器

语法
```
CREATE TRIGGER trigger_name
ON table_name
WHEN condition
EXECUTE FUNCTION function_name(arguments);
```

In [9]:
sql = """
CREATE TABLE orders (
    order_id INT GENERATED ALWAYS AS IDENTITY PRIMARY KEY,
    customer_id INT NOT NULL,
    total_amount NUMERIC NOT NULL DEFAULT 0,
    status VARCHAR(20) NOT NULL
);

CREATE TABLE customer_stats (
    customer_id INT PRIMARY KEY,
    total_spent NUMERIC NOT NULL DEFAULT 0
);

CREATE OR REPLACE FUNCTION insert_customer_stats()
RETURNS TRIGGER 
AS $$
BEGIN
   INSERT INTO customer_stats (customer_id)
   VALUES (NEW.customer_id);
   RETURN NULL;
END;
$$ LANGUAGE plpgsql;

CREATE TRIGGER insert_customer_stats_trigger
AFTER INSERT ON orders
FOR EACH ROW
EXECUTE FUNCTION insert_customer_stats();


CREATE OR REPLACE FUNCTION update_customer_stats()
RETURNS TRIGGER 
AS 
$$
BEGIN
    IF NEW.status = 'completed' THEN
        -- Update the total_spent for the customer
        UPDATE customer_stats
        SET total_spent = total_spent + NEW.total_amount
        WHERE customer_id = NEW.customer_id;
    END IF;
    RETURN NULL;
END;
$$ LANGUAGE plpgsql;

CREATE TRIGGER update_customer_stats_trigger
AFTER UPDATE ON orders
FOR EACH ROW
WHEN (OLD.status <> 'completed' AND NEW.status = 'completed')
EXECUTE FUNCTION update_customer_stats();
"""
cursor.execute(sql)

<psycopg.Cursor [COMMAND_OK] [INTRANS] (host=localhost user=postgres database=dvdrental) at 0x1b4898550d0>

In [10]:
sql = """
INSERT INTO orders (customer_id, total_amount, status)
VALUES
    (1, 100, 'pending'),
    (2, 200, 'pending');
    
UPDATE orders
SET status = 'completed'
WHERE customer_id IN (1,2);
"""
cursor.execute(sql)

<psycopg.Cursor [COMMAND_OK] [INTRANS] (host=localhost user=postgres database=dvdrental) at 0x1b4898550d0>

In [11]:
sql = """
SELECT * FROM customer_stats;
"""
run_sql(cursor, sql)

   customer_id total_spent
0            1         100
1            2         200
